In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import geopandas as gpd
import matplotlib.ticker as mticker
import requests
import zipfile
import io

matplotlib.use('Agg')

In [18]:

import pandas as pd

# Load the data from the uploaded CSV file
file_path = 'Resources/data_interpolated.csv'
data = pd.read_csv(file_path)

# Display the first 5 rows
data.head()

,Date,CityName,StateName,MedianSalePrice,Year,MedianListPrice,SalesTotal,Total Population,Median Income,MORTGAGE30US,Unemployment Rate,HAI
0,2008-02-29,United States,NaN,174000.0,2008.0,NaN,205206.0,NaN,NaN,NaN,NaN,NaN
1,2008-02-29,New York,NY,400000.0,2008.0,NaN,8591.0,NaN,NaN,NaN,NaN,NaN
2,2008-02-29,Los Angeles,CA,470000.0,2008.0,NaN,4159.0,NaN,NaN,NaN,NaN,NaN
3,2008-02-29,Chicago,IL,224000.0,2008.0,NaN,5933.0,NaN,NaN,NaN,NaN,NaN
4,2008-02-29,Dallas,TX,138000.0,2008.0,NaN,5058.0,NaN,NaN,NaN,NaN,NaN


In [19]:
# Clean the data
# Drop rows with NaN values
data.dropna(inplace=True)

# Drop columns with duplicate values
data = data.drop(columns=['Date', 'CityName', 'StateName'])

# Convert relevant columns to numeric data type & round to whole number
numeric_columns = ['Year', 'Median Income', 'MORTGAGE30US', 'MedianSalePrice', 'MedianListPrice', 'SalesTotal', 'Unemployment Rate', 'Total Population']
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce').round().astype(int)

# Rename columns
data.rename(columns={'MORTGAGE30US': 'Mortgage Interest Rate', 
                     'MedianSalePrice': 'Median Sale Price',
                     'MedianListPrice': 'Median List Price', 
                     'SalesTotal': 'Sales Total'}, inplace=True)

# Filter by years 2009 to 2022
data = data[(data['Year'] >= 2018) & (data['Year'] <= 2022)]

# Drop duplicate rows
data = data.drop_duplicates()

data.head()

,Median Sale Price,Year,Median List Price,Sales Total,Total Population,Median Income,Mortgage Interest Rate,Unemployment Rate,HAI
83026,390000,2018,521300,17725,8443713,60762,5,7,79.514731
83027,640000,2018,743333,10466,3959657,58385,5,7,46.558765
83028,240740,2018,300600,16976,2718555,55198,5,9,117.018689
83029,265000,2018,331797,9877,1318806,50100,5,5,96.487709
83030,230000,2018,299633,7993,2295982,51140,5,6,113.478354


In [27]:
# Build function to change secondary x axis displayed
def plot_with_secondary_axis(y_variable):

# Apply the default theme



# Group the filtered data by 'Year' and calculate the mean of 'SalesTotal' and the specified y_variable
    us_data = data.groupby('Year').agg({'Sales Total':'sum', y_variable:'mean'}).reset_index()

# Building sns barplot
    plt.figure(figsize=(12, 8))
    sns.barplot(data=us_data, x=us_data.index, y=us_data['Sales Total'], color='blue', saturation=0.75, fill=True)
    sns.set_style("white")
    sns.color_palette("husl", 9)

# Create a secondary y-axis
    ax2 = plt.gca().twinx()

# Your Seaborn line plot
    sns.lineplot(x=us_data.index, y=y_variable, data=us_data, color='red', ax=ax2)

# Set x-axis ticks and labels
    plt.xticks(ticks=us_data.index, labels=us_data['Year'])

# Set labels and title
    plt.xlabel('Year')
    plt.ylabel('Total Home Sales in US')
    ax2.set_ylabel(y_variable)
    plt.title(f'Total Home Sales and {y_variable} in the US from 2018 to 2022')

# Save the plot to a file
    plt.savefig('seaborn_plot_test.png', facecolor='white', bbox_inches='tight')  # Set background color to white

# Execute function
plot_with_secondary_axis('Median Sale Price')

c:\Users\Adda\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\Adda\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\Adda\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\Adda\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categor

In [ ]:
# Old attempts for reference just in case...

In [ ]:
'''def plot_with_secondary_axis(y_variable):
    # Apply the default theme
    sns.set_theme()

    # Group the filtered data by 'Year' and calculate the mean of 'SalesTotal' and the specified y_variable
    us_data = data.groupby('Year').agg({'SalesTotal':'sum', y_variable:'mean'}).dropna().reset_index()

    # Convert 'Year' column to string type and extract integer part
    us_data['Year'] = us_data['Year'].astype(str).str.split('.').str[0]
    us_data['Year'] = us_data['Year'].astype(int)  # Convert 'Year' to integer
    us_data[y_variable] = us_data[y_variable].astype(int)  # Convert y_variable to integer
    
    # Filter for the years w/ Median Income data and exclude rows where 'CityName' is "United States"
    filtered_data = us_data[(us_data['Year'] >= 2009) & (us_data['Year'] <= 2022)] #& (us_data['CityName'] != 'United States')]

    # Building sns barplot
    plt.figure(figsize=(12, 8))
    sns.barplot(data=filtered_data, x=filtered_data.index, y='SalesTotal', saturation=0.75, fill=True)

    # Create a secondary y-axis
    ax2 = plt.gca().twinx()

    # Your Seaborn line plot
    sns.lineplot(x='Year', y=y_variable, data=filtered_data, ax=ax2)

    # Set labels and title
    plt.xlabel('Year')
    plt.ylabel('Total Home Sales in US')
    ax2.set_ylabel(y_variable)
    plt.title(f'Total Home Sales and {y_variable} in the US Over Years')

    # Save the plot to a file
    plt.savefig('seaborn_plot_test.png')

    # Show the plot
    plt.show()

# Example usage:
# Assuming your DataFrame is named 'data'
plot_with_secondary_axis('MORTGAGE30US')
# Help!'''

In [ ]:
'''def plot_with_secondary_axis(data, y_variable):
    # Apply the default theme
    sns.set_theme()
    
    # Filtering data specifically for the United States and aggregating by year, correcting 'Year' data type
    #data['Year'] = data['Year'].astype(str).str.split('.').str[0]
    # us_data = data[data['CityName'] == 'United States'].groupby('Year').agg({'SalesTotal':'sum', y_variable:'mean'}).dropna()
    us_data = data[data['CityName'] == 'United States'].groupby('Year').agg({'SalesTotal':'sum', y_variable:'mean'}).dropna().reset_index()
    us_data['Year'] = us_data['Year'].astype(int)  # Convert 'Year' to integer

    # Building sns barplot
    plt.figure(figsize=(12, 8))
    sns.barplot(data=us_data, x=us_data.index, y=us_data['SalesTotal'], saturation=0.75, fill=True)

    # Create a secondary y-axis
    ax2 = plt.gca().twinx()

    # Your Seaborn line plot
    sns.lineplot(x=us_data.index, y=us_data[y_variable], data=us_data, ax=ax2)

    # Set labels and title
    plt.xlabel('Year')
    plt.ylabel('Total Home Sales in US')
    ax2.set_ylabel(y_variable)
    plt.title(f'Total Home Sales and {y_variable} in the US Over Years')

    # Save the plot to a file
    plt.savefig('seaborn_plot_test.png')

    # Show the plot
    plt.show()

# Example usage:
# Assuming your DataFrame is named 'data'
plot_with_secondary_axis(data, 'MORTGAGE30US')'''

In [ ]:
'''# Apply the default theme
sns.set_theme()

# Group the filtered data by 'Year' and calculate the mean of 'SalesTotal' and the specified y_variable
us_data = data.groupby('Year').agg({'SalesTotal':'sum', y_variable:'mean'}).reset_index()

# Building sns barplot
plt.figure(figsize=(12, 8))
sns.barplot(data=us_data, x=us_data.index, y=us_data['SalesTotal'], saturation=0.75, fill=True)

# Create a secondary y-axis
ax2 = plt.gca().twinx()

# Your Seaborn line plot
sns.lineplot(x=us_data.index, y=y_variable, data=us_data, ax=ax2)

# Set x-axis ticks and labels
plt.xticks(ticks=us_data.index, labels=us_data['Year'])

# Set labels and title
plt.xlabel('Year')
plt.ylabel('Total Home Sales in US')
ax2.set_ylabel('Mortgage Interest Rate (%)')
plt.title('Total Home Sales and Mortgage Interest Rate in the US Over Years')

# Save the plot to a file
plt.savefig('seaborn_plot_test.png')'''

In [ ]:
'''# Apply the default theme
sns.set_theme()

# Filtering data specifically for the United States and aggregating by year
data['Year'] = data['Year'].astype(str)
us_data = data[data['CityName'] == 'United States'].groupby('Year').agg({'SalesTotal':'sum', 'MORTGAGE30US':'mean'}).dropna()

# Building sns barplot
plt.figure(figsize=(12, 8))
sns.barplot(data=us_data, x=us_data.index, y=us_data['SalesTotal'], saturation=0.75, fill=True)

# Create a secondary y-axis
ax2 = plt.gca().twinx()

# Your Seaborn line plot
sns.lineplot(x=us_data.index, y=us_data['MORTGAGE30US'], data=us_data, ax=ax2)

# Set labels and title
plt.xlabel('Year')
plt.ylabel('Total Home Sales in US')
ax2.set_ylabel('Mortgage Interest Rate (%)')
plt.title('Total Home Sales and Mortgage Interest Rate in the US Over Years')

# Show the plot
plt.show()

# Save the plot to a file
plt.savefig('seaborn_plot_test.png')'''

In [ ]:
'''# Apply the default theme
sns.set_theme()

# Filtering data specifically for the United States and aggregating by year
us_data = data[data['CityName'] == 'United States'].groupby('Year').agg({'SalesTotal':'sum', 'MORTGAGE30US':'mean'}).dropna()

# Creating the combined graph for US data
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar graph for home sales in the US
ax1.bar(us_data.index, us_data['SalesTotal'], color='b', alpha=0.6, label='Total Home Sales in US')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Home Sales in US', color='b')
ax1.tick_params('y', colors='b')

# Format the y-axis to avoid exponential notation
ax1.get_yaxis().set_major_formatter(mticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# Line graph for mortgage interest rate in the US
ax2 = ax1.twinx()
ax2.plot(us_data.index, us_data['MORTGAGE30US'], color='r', label='Mortgage Interest Rate (%)')
ax2.set_ylabel('Mortgage Interest Rate (%)', color='r')
ax2.tick_params('y', colors='r')

# Title and legend
plt.title('Total Home Sales and Mortgage Interest Rate in the US Over Years')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Show plot
plt.savefig('png/Total Home Sales and Mortgage Interest Rate in the US Over Years.png', dpi = 300)
plt.show()'''

In [ ]:
'''sns.barplot(x='Category', y='Value', data=df)
plt.title('Bar Chart Example')
plt.xlabel('Category')
plt.ylabel('Value')
plt.show()'''